# Find factors to identify XBI bottom and top

In [49]:
from yahooquery import Ticker
import pandas as pd
from datetime import datetime
import os
# USING MINI for Testing
tickers = pd.read_excel(r"c:\Users\MukeshwaranBaskaran\Downloads\short_alpha\Complete-List-of-Biotech-Stocks-Listed-on-NASDAQ-Jan-1-24(1) - Copy.xlsx")
tickers = tickers[tickers.Country == "United States"]
tickers = tickers.Ticker.unique().tolist()
# USING MINI for Testing
tickers_mini = tickers[:10]

benchmark_tickers = ["XBI","SPY"]
tickers_full = tickers_mini + benchmark_tickers

# EXAMPLE USAGE yahooquery ; https://yahooquery.dpguthrie.com/guide/ticker/historical/
# ticker_list = ["A", "AL", "AAP", "AAPL", "ZBRA", "ZION", "ZTS"]
# all_symbols = " ".join(ticker_list)
# myInfo = Ticker(all_symbols)
# myDict = myInfo.price

# for ticker in ticker_list:
#     ticker = str(ticker)
#     longName = myDict[ticker]['longName']
#     market_cap = myDict[ticker]['marketCap']
#     price = myDict[ticker]['regularMarketPrice']
#     print(ticker, longName, market_cap, price)

def create_folder_if_not_exists(folder_name):
    """Create folder if it doesn't already exist."""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

def fetch_ticker_data(tickers, period='ytd', interval='1d', filename=None):
    """Fetch historical data for the given tickers and save it as a CSV file."""
    try:
        tickers_obj = Ticker(tickers, asynchronous=True)
        df = tickers_obj.history(period=period, interval=interval)
        df.reset_index(inplace=True)
        
        # Save to CSV if filename is provided
        if filename:
            df.to_csv(filename, index=False)
        
        return df
    except Exception as e:
        print(f"Error fetching data for {tickers}: {e}")
        return None

def save_data_to_csv(df, folder_name, file_name):
    """Save the DataFrame to a CSV file in the specified folder."""
    create_folder_if_not_exists(folder_name)
    file_path = os.path.join(folder_name, file_name)
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

# Define tickers and folder
folder_name = 'Input'

# Fetch daily data for the year-to-date (YTD)
df_daily_ticker = fetch_ticker_data(tickers_full, period='max', interval='1d', filename=None)
save_data_to_csv(df_daily_ticker, folder_name, 'daily_data.csv')

# Fetch today's minute-level data
df_today_minute = fetch_ticker_data(tickers_full, period='1d', interval='1m', filename=None)
save_data_to_csv(df_today_minute, folder_name, 'minute_data.csv')

c:\Users\MukeshwaranBaskaran\Miniconda3\envs\conda_env\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\MukeshwaranBaskaran\Miniconda3\envs\conda_env\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\MukeshwaranBaskaran\Miniconda3\envs\conda_env\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\MukeshwaranBaskaran\Miniconda3\envs\conda_env\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and wil

Data saved to Input\daily_data.csv
Data saved to Input\minute_data.csv


In [ ]:
cols_needed_daily = ['symbol', 'date', 'open', 'high', 'low', 'adjclose', 'volume']
df_ticker_daily = pd.read_csv("INPUT\daily_data.csv")
df_ticker_daily = df_ticker_daily[cols_needed_daily]
df_ticker_daily[df_ticker_daily.symbol == "XBI"]

df_ticker_minute = pd.read_csv("INPUT\minute_data.csv")

In [63]:
import backtrader as bt
import pandas as pd
from xbi_signals import XBISignalGenerator, load_and_prepare_data

class XBISignalStrategy(bt.Strategy):
    params = (
        ('signal_threshold', 0.5),
    )
    
    def __init__(self, signals):
        """
        Initialize strategy with pre-computed signals
        
        Args:
            signals (pd.DataFrame): Signals DataFrame
        """
        self.signals = signals
        self.signal_index = 0
        
        # Performance tracking
        self.trades = []
    
    def next(self):
        # Ensure we're within signal DataFrame
        if self.signal_index >= len(self.signals):
            return
        
        current_signals = self.signals.iloc[self.signal_index]
        
        # Buy signal logic
        if current_signals['composite_buy_signal'] > self.p.signal_threshold:
            if not self.position:
                self.buy()
                self.trades.append(('buy', self.data.close[0]))
        
        # Sell signal logic
        elif current_signals['composite_sell_signal'] > self.p.signal_threshold:
            if self.position:
                self.close()
                self.trades.append(('sell', self.data.close[0]))
        
        self.signal_index += 1
    
    def stop(self):
        """Log trades at end of backtest"""
        print("Trades:", self.trades)

class XBIBacktest:
    def __init__(self, data_path):
        """
        Initialize backtest with data
        
        Args:
            data_path (str): Path to financial data
        """
        self.df = load_and_prepare_data(data_path)
        self.signal_generator = XBISignalGenerator(self.df)
    
    def run_backtest(self, initial_cash=100000, commission=0.001):
        """
        Run comprehensive backtest
        
        Args:
            initial_cash (float): Starting capital
            commission (float): Trading commission rate
        
        Returns:
            bt.Cerebro: Backtrader cerebro instance
        """
        # Generate signals
        signals = self.signal_generator.get_final_signals()
        
        # Prepare Backtrader
        cerebro = bt.Cerebro()
        
        # Convert DataFrame to Backtrader data feed
        data = bt.feeds.PandasData(dataname=self.df)
        cerebro.adddata(data)
        
        # Add strategy with signals
        cerebro.addstrategy(XBISignalStrategy, signals=signals)
        
        # Broker setup
        cerebro.broker.setcash(initial_cash)
        cerebro.broker.setcommission(commission=commission)
        
        # Performance analyzers
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
        
        # Run backtest
        results = cerebro.run()
        
        # Performance reporting
        strategy = results[0]
        print("Final Portfolio Value: $", cerebro.broker.getvalue())
        
        return cerebro
    
df_xbi = df_ticker_daily[df_ticker_daily.symbol == "XBI"]
backtest = XBIBacktest(df_xbi)
cerebro = backtest.run_backtest()
cerebro.plot()


TypeError: argument of type 'method' is not iterable